In [1]:
from math import sqrt

M = 101
N = 1000001
assert N < 3628800
SQRTN = int(sqrt(N))
INFINITY = 2 * N

class info:
    sqrt = "√"
    def __init__(self, expr = "", result = None, minlen = INFINITY, lastop = "", first = None, second = None, evalu = None):
        if isinstance(expr, info):
            self.expr = expr.expr
            self.result = expr.result
            self.minlen = expr.minlen
            self.lastop = expr.lastop
            self.first = expr.first
            self.second = expr.second
            self.evalu = expr.evalu
        else:
            self.expr = expr
            self.result = result
            self.minlen = minlen
            self.lastop = lastop
            self.first = first
            self.second = second
            self.evalu = evalu
    def __str__(self):
        return f'expr="{self.expr}", result={self.result}, minlen={self.minlen}, lastop="{self.lastop}", first={self.first}, second={self.second}'

def evaluation_length(infobj):
    if infobj.evalu is None:
        assert infobj.expr
        infobj.evalu = len(infobj.expr)
    return infobj.evalu

def evaluation_mine(infobj):
    # 10   + -
    # 15   * /
    # 35   ^
    # 50   ! √
    # 80  ()
    # 90 exceed number
    if infobj.evalu is None:
        assert infobj.expr
        infobj.evalu =  10 * (infobj.expr.count("+") + infobj.expr.count("-"))
        infobj.evalu += 15 * (infobj.expr.count("*") + infobj.expr.count("/"))
        infobj.evalu += 35 * infobj.expr.count("^")
        infobj.evalu += 50 * (infobj.expr.count("√") + infobj.expr.count("!"))
        infobj.evalu += 80 * infobj.expr.count("(")
#         if not (infobj.first is None or isinstance(infobj.first, tuple)) and infobj.result:
#             infobj.evalu += 90 * max(0, infobj.first - infobj.result) / infobj.result
#         if not (infobj.second is None) and infobj.result:
#             infobj.evalu += 90 * max(0, infobj.second - infobj.result) / infobj.result
    return infobj.evalu

evaluate = evaluation_mine

def print_infos():
    global infos
    global M
    print("infos = [[],")
    for digit in range(1, 10):
        print("        ")
        for i in range(M):
            obj = infos[digit][i]
            if i == 0:
                print('        [info(', end="")
            else:
                print('        info(', end="")
            if i == M-1:
                if digit == 9:
                    print(f'"{obj.expr}", {obj.minlen}, "{obj.lastop}", {obj.first}, {obj.second})]]')
                else:
                    print(f'"{obj.expr}", {obj.minlen}, "{obj.lastop}", {obj.first}, {obj.second})],')
            else:
                print(f'"{obj.expr}", {obj.minlen}, "{obj.lastop}", {obj.first}, {obj.second}),')

squares = [0, 1, 4]
i = 3
while i * i < N:
    squares.append(i * i)
    i += 1

sqrtdict = {4 : 2}
i = 3
while i * i < N:
    sqrtdict[i * i] = i
    i += 1

antifact = {6:3}
i = 4
j = 24
while j < N:
    antifact[j] = i
    i += 1
    j *= i

powers = dict()
for n in range(2, SQRTN + 2):
    m = 2
    while n ** m < N:
        if powers.get(n ** m):
            powers[n**m].append((n, m))
        else:
            powers[n**m] = [(n, m)]
        m += 1

factorials = [1, 1, 2]
i = 3
while factorials[-1] * i < N:
    factorials.append(factorials[-1] * i)
    i += 1

maxy = [1 for i in range(N)]
while 2 ** maxy[2] < N:
    maxy[2] += 1
maxy[2] -= 1
curx = 3
while maxy[curx-1] > 1:
    maxy[curx] = maxy[curx-1]
    while curx ** maxy[curx] >= N:
        maxy[curx] -= 1
    curx += 1
maxx = curx - 2
maxy[1] = N-1




infos = [[info()]]
num = [[set() for i in range(len(str(N)))] for digit in range(10)]
for digit in range(1, 10):
    infos.append([info() for _ in range(N)])
    i = digit
    j = 1
    while i < N:
        infos[digit][i] = info(str(i), i, j)
        num[digit][j].add(i)
        i = i * 10 + digit
        j += 1

for digit in range(1, 10):
    curr = infos[digit]
    step = 0
    while max([infobj.minlen for infobj in curr[:M]]) == INFINITY:
        step += 1
        if len(num[digit]) == step:
            num[digit].append(set())
        # +, -, *
        for it in range(1, step // 2 + 1):
            for x in num[digit][it]:
                for y in num[digit][step-it]:
                    # Addition
                    buff = x + y
                    if buff < N and curr[buff].minlen >= step:
                        old = info(curr[buff])
                        curr[buff] = info(curr[x].expr + "+" + curr[y].expr, buff, step, "+", x, y)
                        num[digit][step].add(buff)
                        if old.minlen == step and evaluate(old) < evaluate(curr[buff]):
                            curr[buff] = old
                        
                    
                    # Subtraction
                    buff, bx, by = abs(x-y), max(x, y), min(x, y)
                    if curr[buff].minlen >= step:
                        old = info(curr[buff])
                        curr[buff] = info(curr[bx].expr + "-", buff, step, "-", bx, by)
                        if curr[by].lastop in {"+", "-"}:
                            curr[buff].expr += "(" + curr[by].expr + ")"
                        else:
                            curr[buff].expr += curr[by].expr
                        num[digit][step].add(buff)
                        if old.minlen == step and evaluate(old) < evaluate(curr[buff]):
                            curr[buff] = old
                    
                    # Multiplication
                    buff = x * y
                    if buff < N and curr[buff].minlen >= step:
                        old = info(curr[buff])
                        curr[buff] = info("", buff, step, "*", x, y)
                        if curr[x].lastop in {"+", "-"}:
                            curr[buff].expr = "(" + curr[x].expr + ")*"
                        else:
                            curr[buff].expr = curr[x].expr + "*"
                        if curr[y].lastop in {"+", "-"}:
                            curr[buff].expr += "(" + curr[y].expr + ")"
                        else:
                            curr[buff].expr += curr[y].expr
                        num[digit][step].add(buff)
                        if old.minlen == step and evaluate(old) < evaluate(curr[buff]):
                            curr[buff] = old
        
        # ^
        for it in range(1, step):
            for x in (num[digit][it] - {0}):
                if x <= maxx:
                    for y in num[digit][step-it]:
                        if y <= maxy[x]:
                            
                            # Exponentiation
                            buff = x ** y
                            if curr[buff].minlen >= step:
                                old = info(curr[buff])
                                curr[buff] = info("", buff, step, "^", x, y)
                                if curr[x].lastop in {"+", "-", "*", "/"}:
                                    curr[buff].expr = "(" + curr[x].expr + ")^"
                                else:
                                    curr[buff].expr = curr[x].expr + "^"
                                if curr[y].lastop in {"+", "-", "*", "/", "^"}:
                                    curr[buff].expr += "(" + curr[y].expr + ")"
                                else:
                                    curr[buff].expr += curr[y].expr
                                num[digit][step].add(buff)
                                if old.minlen == step and evaluate(old) < evaluate(curr[buff]):
                                    curr[buff] = old
        
        # /
        for it in range(1, step):
            for x in num[digit][it]:
                for y in num[digit][step-it] - {0}:
                    
                    # Division
                    buff = x // y
                    if (not x % y) and curr[buff].minlen >= step:
                        old = info(curr[buff])
                        curr[buff] = info("", buff, step, "/", x, y)
                        if curr[x].lastop in {"+", "-"}:
                            curr[buff].expr = "(" + curr[x].expr + ")/"
                        else:
                            curr[buff].expr = curr[x].expr + "/"
                        if curr[y].lastop in {"+", "-", "*", "/"}:
                            curr[buff].expr += "(" + curr[y].expr + ")"
                        else:
                            curr[buff].expr += curr[y].expr
                        num[digit][step].add(buff)
                        if old.minlen == step and evaluate(old) < evaluate(curr[buff]):
                            curr[buff] = old
        
        # Miracle type 1: sqrt √
        for it in range(1, step):
            for x in (num[digit][it] - {0,1}):
                if x <= maxx:
                    for y in (num[digit][step-it] - {0}):
                        p = y
                        k = 0
                        while p > maxy[x] and not (p % 2):
                            p //= 2
                            k += 1
                        if not k and not (p % 2):
                            p //= 2
                            k = 1
                        while not (p % 2):
                            
                            # Miracle
                            buff = x ** p
                            if curr[buff].minlen >= step:
                                old = info(curr[buff])
                                curr[buff] = info("", buff, step, "^", (x, k), y)
                                if curr[x].lastop in {"+", "-", "*", "/", "^", "!"}:
                                    curr[buff].expr = "√" * k + "(" + curr[x].expr + ")^"
                                else:
                                    curr[buff].expr = "√" * k + curr[x].expr + "^"
                                if curr[y].lastop in {"+", "-", "*", "/", "^"}:
                                    curr[buff].expr += "(" + curr[y].expr + ")"
                                else:
                                    curr[buff].expr += curr[y].expr
                                num[digit][step].add(buff)
                                if old.minlen == step and evaluate(old) < evaluate(curr[buff]):
                                    curr[buff] = old
                            p //= 2
                            k += 1
        
        # !, √
        setcurr = num[digit][step]
        while setcurr:
            setnew = set()
            for x in setcurr:
                if x < len(factorials) and x > 2:
                    
                    # Factorial
                    buff = factorials[x]
                    if curr[buff].minlen >= step:
                        old = info(curr[buff])
                        curr[buff] = info("", buff, step, "!", x)
                        if curr[x].lastop in {"+", "-", "*", "/", "^"}:
                            curr[buff].expr = "(" + curr[x].expr + ")!"
                        else:
                            curr[buff].expr = curr[x].expr + "!"
                        if old.minlen > step:
                            setnew.add(buff)
                        elif evaluate(old) < evaluate(curr[buff]):
                            curr[buff] = old
                if sqrtdict.get(x) and x > 2:
                    
                    # Square Root
                    buff = sqrtdict[x]
                    if curr[buff].minlen >= step:
                        old = info(curr[buff])
                        curr[buff] = info("", sqrtdict[x], step, "√", x)
                        if curr[x].lastop in {"+", "-", "*", "/", "^", "!"}:
                            curr[buff].expr = "√(" + curr[x].expr + ")"
                        else:
                            curr[buff].expr = "√" + curr[x].expr
                        if old.minlen > step:
                            setnew.add(buff)
                        elif evaluate(old) < evaluate(curr[buff]):
                            curr[buff] = old
            num[digit][step] |= setnew
            setcurr = setnew
        
    print(f"DIGIT {digit} IS DONE ----------------------------------------------------------- ")

DIGIT 1 IS DONE ----------------------------------------------------------- 
DIGIT 2 IS DONE ----------------------------------------------------------- 
DIGIT 3 IS DONE ----------------------------------------------------------- 
DIGIT 4 IS DONE ----------------------------------------------------------- 
DIGIT 5 IS DONE ----------------------------------------------------------- 
DIGIT 6 IS DONE ----------------------------------------------------------- 
DIGIT 7 IS DONE ----------------------------------------------------------- 
DIGIT 8 IS DONE ----------------------------------------------------------- 
DIGIT 9 IS DONE ----------------------------------------------------------- 
